In [1]:
import requests
from bs4 import BeautifulSoup

# URL de la fiche de film
url = 'https://www.jpbox-office.com/fichfilm.php?id=15278'

# Envoyer une requête HTTP GET à l'URL
response = requests.get(url)

# Analyser le contenu HTML de la page avec Beautiful Soup
soup = BeautifulSoup(response.text, 'html.parser')

# Exemple d'extraction du titre du film
# Ceci est un exemple générique et peut ne pas fonctionner tel quel sans adaptation
titre = soup.find('h1').text

print(f'Titre du film: {titre}')

Titre du film: 
                            Mon roi


In [3]:
# URL de la fiche film
url = 'https://www.jpbox-office.com/fichfilm.php?id=15278'

# Envoyer une requête HTTP GET à l'URL
response = requests.get(url)

# Parser le contenu HTML de la page avec BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Rechercher les données spécifiques : titre et budget
# Remarque : Ces sélecteurs dépendent fortement de la structure du site et peuvent nécessiter des ajustements.
titre = soup.find('h1').text
budget_tag = soup.find(text='Budget').findNext('td')

# Afficher le titre et le budget
print(f"Titre: {titre}")
if budget_tag:
    print(f"Budget: {budget_tag.text}")
else:
    print("Budget non trouvé")

Titre: 
                            Mon roi
Budget: 9 473 556 €


C:\Users\maoui\AppData\Local\Temp\ipykernel_9352\3263200215.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  budget_tag = soup.find(text='Budget').findNext('td')


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Initialisation du DataFrame
df_films = pd.DataFrame(columns=['ID', 'Titre', 'Budget'])

# Définition de la plage des ID des films à scraper
id_debut = 15270
id_fin = 15280

# Boucle à travers les ID de films
for film_id in range(id_debut, id_fin + 1):
    url = f'https://www.jpbox-office.com/fichfilm.php?id={film_id}'
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extraction du titre
            titre_element = soup.find('h1')
            titre_film = titre_element.text.strip() if titre_element else 'Titre inconnu'
            
            # Tentative d'extraction du budget
            try:
                # Utilisation de string au lieu de text pour éviter l'avertissement de dépréciation
                budget_tag = soup.find(string='Budget').find_next('td')
                budget_film = budget_tag.text if budget_tag else 'Budget inconnu'
            except AttributeError:
                # Gestion du cas où 'Budget' n'est pas trouvé
                budget_film = 'Budget inconnu'
            
            # Ajout des données au DataFrame
            df_temp = pd.DataFrame([[film_id, titre_film, budget_film]], columns=['ID', 'Titre', 'Budget'])
            df_films = pd.concat([df_films, df_temp], ignore_index=True)
    except Exception as e:
        print(f"Erreur lors du scraping de l'ID {film_id}: {e}")

# Affichage du DataFrame pour vérification
print(df_films)


       ID                                              Titre          Budget
0   15270                         Qui c'est les plus forts ?     6 227 650 €
1   15271                                     Valley of Love     2 900 626 €
2   15272                                  Daddy Cool (2015)     6 700 000 $
3   15273  Love & Mercy, la vÃ©ritable histoire de Brian ...             ? $
4   15274                                          11th Hour  Budget inconnu
5   15275                              Set Fire to the Stars             ? $
6   15276                                     MacBeth (2015)             ? $
7   15277                                            Dheepan     7 984 811 €
8   15278                                            Mon roi     9 473 556 €
9   15279                                              Carol    11 800 000 $
10  15280                                       The Assassin    14 900 000 $


In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re  # Import du module pour les expressions régulières

# Initialisation du DataFrame
df_films = pd.DataFrame(columns=['ID', 'Titre Local', 'Titre International', 'Budget'])

# Définition de la plage des ID des films à scraper
id_debut = 15270
id_fin = 15280

# Boucle à travers les ID de films
for film_id in range(id_debut, id_fin + 1):
    url = f'https://www.jpbox-office.com/fichfilm.php?id={film_id}'
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extraction et traitement du titre à partir de l'élément <title>
            title_element = soup.head.find('title')
            title_text = title_element.text if title_element else ''
            
            # Utilisation d'une expression régulière pour extraire le titre local et international
            match = re.match(r"(.+?) \((.+?)\) \(\d{4}\)", title_text)
            if match:
                titre_local = match.group(1)
                titre_international = match.group(2)
            else:
                titre_local = title_text.split(' - ')[0]  # Fallback sur le titre complet sans traitement
                titre_international = 'Titre international inconnu'
            
            # Tentative d'extraction du budget
            try:
                budget_tag = soup.find(string='Budget').find_next('td')
                budget_film = budget_tag.text.strip() if budget_tag else 'Budget inconnu'
            except AttributeError:
                budget_film = 'Budget inconnu'
            
            # Ajout des données au DataFrame
            df_temp = pd.DataFrame([[film_id, titre_local, titre_international, budget_film]], columns=['ID', 'Titre Local', 'Titre International', 'Budget'])
            df_films = pd.concat([df_films, df_temp], ignore_index=True)
    except Exception as e:
        print(f"Erreur lors du scraping de l'ID {film_id}: {e}")

# Affichage du DataFrame pour vérification
print(df_films)


       ID                         Titre Local          Titre International  \
0   15270  Qui c\'est les plus forts ? (2015)  Titre international inconnu   
1   15271               Valley of Love (2015)  Titre international inconnu   
2   15272        Infinitely Polar Bear (2015)  Titre international inconnu   
3   15273                 Love & Mercy (2015)  Titre international inconnu   
4   15274                    11th Hour (2015)  Titre international inconnu   
5   15275        Set Fire to the Stars (2015)  Titre international inconnu   
6   15276                             MacBeth                         2015   
7   15277                      Dheepan (2015)  Titre international inconnu   
8   15278                             Mon roi                      My King   
9   15279                        Carol (2015)  Titre international inconnu   
10  15280                       Nie yin niang                 The Assassin   

            Budget  
0      6 227 650 €  
1      2 900 626 €  


In [19]:
df_films.to_csv(r"C:\Users\maoui\OneDrive\Documents\Data Scientest\DTS - Projet\jpboxoffice.csv", index=False)

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re  # Utiliser les expressions régulières pour extraire les titres et années

# Initialisation du DataFrame
df_films = pd.DataFrame(columns=['ID', 'Titre Local', 'Titre International', 'Budget', 'Annee'])

# Définition de la plage des ID des films à scraper
id_debut = 15270
id_fin = 15280

# Boucle à travers les ID de films
for film_id in range(id_debut, id_fin + 1):
    url = f'https://www.jpbox-office.com/fichfilm.php?id={film_id}'
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extraction et traitement du titre à partir de l'élément <title>
            title_element = soup.head.find('title')
            title_text = title_element.text if title_element else ''
            
            # Utilisation d'une expression régulière pour extraire le titre local, international et l'année
            match = re.search(r'(.+?) \((.*?)\) \((\d{4})\)', title_text)
            if match:
                titre_local = match.group(1)
                titre_international = match.group(2) if match.group(2) else 'Titre international inconnu'
                annee = match.group(3)
            else:
                titre_local = title_text.split(' - ')[0]  # Fallback sur le titre complet sans traitement
                titre_international = 'Titre international inconnu'
                annee = 'Année inconnue'
            
            # Tentative d'extraction du budget
            budget_tag = soup.find('td', string=re.compile('Budget'))
            budget_film = budget_tag.find_next('td').text.strip() if budget_tag else 'Budget inconnu'
            budget_film = budget_film.replace(u'\xa0', u' ').encode('ascii', 'ignore').decode('ascii')
            
            # Ajout des données au DataFrame
            df_temp = pd.DataFrame([[film_id, titre_local, titre_international, budget_film, annee]], columns=['ID', 'Titre Local', 'Titre International', 'Budget', 'Annee'])
            df_films = pd.concat([df_films, df_temp], ignore_index=True)
    except Exception as e:
        print(f"Erreur lors du scraping de l'ID {film_id}: {e}")

# Affichage du DataFrame pour vérification
print(df_films)


       ID                         Titre Local          Titre International  \
0   15270  Qui c\'est les plus forts ? (2015)  Titre international inconnu   
1   15271               Valley of Love (2015)  Titre international inconnu   
2   15272        Infinitely Polar Bear (2015)  Titre international inconnu   
3   15273                 Love & Mercy (2015)  Titre international inconnu   
4   15274                    11th Hour (2015)  Titre international inconnu   
5   15275        Set Fire to the Stars (2015)  Titre international inconnu   
6   15276                             MacBeth                         2015   
7   15277                      Dheepan (2015)  Titre international inconnu   
8   15278                             Mon roi                      My King   
9   15279                        Carol (2015)  Titre international inconnu   
10  15280                       Nie yin niang                 The Assassin   

            Budget           Annee  
0       6 227 650   Année 